1) Only use 'Salnty', 'STheta' for predictors (same as HW #1)
2) Remove NaN / NA values from dataset (prior to building train/test sets) (same as HW 3) 

3) Scale all features to improve convergence. Feel free to use the sklearn tool "StandardScaler" - more info here: http://scikit-lear

4) Try mini batch sizes of 50, 2000 and 10,000. Comment on the prediction accuracy base For the calculation of the gradient we will use partial derivative from the text. Pleas Example code from chapter:
derived_gradient = 2/m * X_b.T.dot(X_b.dot(theta) - y)

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression as lm
import sklearn.metrics as metrics
import sklearn.preprocessing as preprocessing

## Step 1 & 2

In [4]:
bottle_df = pd.read_csv("bottle.csv", low_memory=False)

bottle_df.dropna(subset=["Salnty", "STheta", "T_degC"], inplace=True)

X_train, X_test, y_train, y_test = train_test_split(
    bottle_df[["Salnty", "STheta"]],
    bottle_df["T_degC"], test_size=.20,
    random_state=0)

## Step 3

In [6]:
scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
y_train = np.asarray(y_train).reshape((len(y_train), 1))

X_test = scaler.transform(X_test)
y_test = np.asarray(y_test).reshape((len(y_test), 1))

X_train = np.c_[np.ones((len(X_train), 1)), X_train]
X_test = np.c_[np.ones((len(X_test), 1)), X_test]

# Step 4

In [14]:
theta_path_mgd = []
best_thetas = []
m = len(X_train)
eta = 0.1
n_iterations = 100
minibatch_size = [50, 2000, 10000]

np.random.seed(42)
theta = np.random.randn(3, 1)  # random initialization

for size in minibatch_size:
    for epoch in range(n_iterations):
        shuffled_indices = np.random.permutation(m)
        X_b_shuffled = X_train[shuffled_indices]
        y_shuffled = y_train[shuffled_indices]
        for i in range(0, m, size):
            xi = X_b_shuffled[i:i + size]
            yi = y_shuffled[i:i + size]
            gradients = 2 / size * xi.T.dot(xi.dot(theta) - yi)
            theta = theta - eta * gradients
            theta_path_mgd.append(theta)
    best_thetas.append(theta)

In [15]:
for i in range(len(best_thetas)):
    print(f'minibatch size: {minibatch_size[i]}')
    print(f'Coefficients: {best_thetas[i]}')
    print("\n")
    print("Holdout mean squared error: %.2f"
          % metrics.mean_squared_error(y_test, X_test.dot(best_thetas[i])))
    print("Holdout explained variance: %.2f"
          % metrics.explained_variance_score(y_test, X_test.dot(best_thetas[
                                                                    i])))
    print("Holdout r-squared: %.2f" % metrics.r2_score(y_test,
                                                       X_test.dot(
                                                           best_thetas[i])))
    print("\n")

minibatch size: 50
Coefficients: [[10.84495525]
 [ 1.45452721]
 [-6.04653521]]


Holdout mean squared error: 0.23
Holdout explained variance: 0.99
Holdout r-squared: 0.99


minibatch size: 2000
Coefficients: [[10.83836563]
 [ 1.36951838]
 [-5.98466765]]


Holdout mean squared error: 0.23
Holdout explained variance: 0.99
Holdout r-squared: 0.99


minibatch size: 10000
Coefficients: [[10.85797046]
 [-0.59057347]
 [-0.42558493]]


Holdout mean squared error: 13.14
Holdout explained variance: 0.26
Holdout r-squared: 0.26


